# An Experimental Approach to the Monty Hall Problem

### Utilities

In [57]:
/// Helper function to nicely print out F# values
let print x = printfn "%A" x

Collection of utility functions to handle random numbers and selections. If you prefer the simulations to be deterministic, provide a seed for `Random()`. For example, `let random = Random(123)`. In such a case, you need to select **Run All** and not just run individual cells.

In [58]:
open System

let random = Random()

let randomNumber lower higher =
    random.Next(lower, higher + 1)

let randomIndex length =
    random.Next(0, length)

let randomIndexExcept length exceptions =
    let processedExceptions =
        exceptions
        |> List.sort
        |> List.distinct
        |> List.filter (fun x -> 0 <= x || x < length)
    let indices =
        [0..length-1]
        |> List.filter (fun x -> not (List.contains x processedExceptions))
    indices[randomIndex (List.length indices)]

In [59]:
randomNumber 1 100

45

In [60]:
randomIndexExcept 3 []

2

In [61]:
randomIndexExcept 3 [1; 1]

0

In [62]:
randomIndexExcept 3 [0; 2]

1

### Types

In [63]:
/// Represents the outcome of a given door. In other words, the outcome is what is behind the door.
type Outcome =
    | Car
    | Goat

/// Represents a strategy after selecting an initial door choice and then presented with the option
/// to switch after losing doors have been opened.
type Strategy =
    | KeepOriginalChoice
    | Switch

/// Represents the state of a door, whether it has been opened or closed
type State =
    | Open
    | Closed

/// Represents a door in the game
type Door = Outcome * State

/// Represents the result of a game. If the player ends up with the door with a car behind it, the
/// player wins. Otherwise, they lose.
type GameResult =
    | Win
    | Lose

### Simulating the Monty Hall problem

In [64]:
/// Generates 3 doors with only one winning door and all doors closed
let generateDoors () =
    let winningDoor = randomNumber 1 3
    let winningDoorIndex = winningDoor - 1
    let initializer = function
        | index when index = winningDoorIndex -> (Car, Closed)
        | _                                   -> (Goat, Closed)
    List.init 3 initializer

/// Given a player's choice, opens up one of the remaining doors that is not a winning door.
/// The door choice is a 1-indexed integer, i.e., the doors are numbered 1 through 3.
let openNonWinningDoor playerDoorChoice (doors: Door list) : Door list =
    let numberOfDoors = List.length doors
    let playerDoorChoiceIndex = playerDoorChoice - 1
    let winningDoorIndex = List.findIndex (fun (outcome, _) -> outcome = Car) doors
    let doorToReveal = randomIndexExcept numberOfDoors [playerDoorChoiceIndex; winningDoorIndex]
    let handleDoor index (outcome, _state) =
        match index with
        | x when x = playerDoorChoiceIndex -> (outcome, Closed) // leave the player's door closed
        | x when x = winningDoorIndex      -> (outcome, Closed) // leave the winning door closed
        | x when x = doorToReveal          -> (outcome, Open) // open up one of the doors that isn't the player's door and isn't the winning door
        | _ -> (outcome, Closed) // all other doors, if any, should be closed
    List.mapi handleDoor doors

In [65]:
generateDoors () |> print

[(Car, Closed); (Goat, Closed); (Goat, Closed)]


In [66]:
openNonWinningDoor 2 [
    (Car, Closed)  // Door 1: winning door
    (Goat, Closed) // Door 2: player's door
    (Goat, Closed) // Door 3: the door Monty will open in this case
]
|> print

[(Car, Closed); (Goat, Closed); (Goat, Open)]


In [67]:
/// Simulate the Monty Hall game given the player's initial door choice and strategy of either
/// staying with their original choice or switching after one of the non-winning doors are opened.
/// The door choice is a 1-indexed integer, i.e., the doors are numbered 1 through 3.
let simulate doorChoice strategy =
    let doorsAfterOpeningNonWinningChoices =
        generateDoors ()
        |> openNonWinningDoor doorChoice
    let indexOfNonPlayerChoiceDoor =
        doorsAfterOpeningNonWinningChoices
        |> List.indexed
        |> List.findIndex (fun (index, (_outcome, state)) ->
            match state with
            | Closed when index <> doorChoice - 1 -> true
            | _                                   -> false)
    let finalDoor =
        match strategy with
        | KeepOriginalChoice -> doorsAfterOpeningNonWinningChoices[doorChoice - 1]
        | Switch             -> doorsAfterOpeningNonWinningChoices[indexOfNonPlayerChoiceDoor]
    match finalDoor with
    | (Car, _)  -> Win
    | (Goat, _) -> Lose

In [68]:
/// Run the given number of simulations of the Monty Hall problem with the given strategy
let runSimulations numberOfSimulations strategy =
    [for _ in 1..numberOfSimulations -> simulate (randomNumber 1 3) strategy]
    |> List.filter (fun result -> result = Win)
    |> List.length
    |> (fun x -> (float x) / (float numberOfSimulations))

In [69]:
runSimulations 1_000_000 Switch

0.667077

In [70]:
runSimulations 1_000_000 KeepOriginalChoice

0.333742